In [116]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [132]:
train = pd.read_csv("../input/titanic/train.csv")
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [117]:
train = pd.read_csv("../input/titanic/train.csv")
data = data.dropna()
y = data.Survived
X = data.drop(["PassengerId", "Name", "Ticket","Cabin", "Survived"], axis=1)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
train_X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
599,1,male,49.0,1,0,56.9292,C
544,1,male,50.0,1,0,106.4250,C
118,1,male,24.0,0,1,247.5208,C
689,1,female,15.0,0,1,211.3375,S
662,1,male,47.0,0,0,25.5875,S
...,...,...,...,...,...,...,...
512,1,male,36.0,0,0,26.2875,S
336,1,male,29.0,1,0,66.6000,S
577,1,female,39.0,1,0,55.9000,S
268,1,female,58.0,0,1,153.4625,S


In [118]:
s = (train_X.dtypes == 'object')
object_cols = list(s[s].index)    
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_X[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(val_X[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_X.index
OH_cols_valid.index = val_X.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_X.drop(object_cols, axis=1)
num_X_valid = val_X.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)
    

In [119]:
data_dmatrix = xgb.DMatrix(data=OH_X_train,label=train_y)

In [120]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [121]:
xg_reg.fit(OH_X_train,train_y)

preds = xg_reg.predict(OH_X_valid)

[22:17:14] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


In [122]:
rmse = np.sqrt(mean_squared_error(val_y, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.425028


In [123]:
binary = [round(x) for x in preds]
accuracy_score(val_y, binary)


0.8478260869565217

In [124]:
test = pd.read_csv("../input/titanic/test.csv")
X = test.drop(["PassengerId", "Name","Ticket","Cabin"], axis=1)
s = (X.dtypes == 'object')
object_cols = list(s[s].index)    
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X[object_cols]))
OH_cols_train.index = X.index
num_X_train = X.drop(object_cols, axis=1)
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
preds = xg_reg.predict(OH_X_train)
binary = [round(x) for x in preds]
tmp = test["PassengerId"]
bin_df = pd.DataFrame(binary)
bin_df
submission = pd.concat([tmp, bin_df], axis=1)


In [125]:
a = test.drop(["PassengerId","Name","Ticket","Cabin"], axis=1)
a

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500,S
414,1,female,39.0,0,0,108.9000,C
415,3,male,38.5,0,0,7.2500,S
416,3,male,NaN,0,0,8.0500,S


In [126]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)    
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(a[object_cols]))
OH_cols_train.index = a.index
num_X_train = a.drop(object_cols, axis=1)
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_train

,Pclass,Age,SibSp,Parch,Fare,0,1,2,3,4
0,3,34.5,0,0,7.8292,0.0,1.0,0.0,1.0,0.0
1,3,47.0,1,0,7.0000,1.0,0.0,0.0,0.0,1.0
2,2,62.0,0,0,9.6875,0.0,1.0,0.0,1.0,0.0
3,3,27.0,0,0,8.6625,0.0,1.0,0.0,0.0,1.0
4,3,22.0,1,1,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
413,3,NaN,0,0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1,39.0,0,0,108.9000,1.0,0.0,1.0,0.0,0.0
415,3,38.5,0,0,7.2500,0.0,1.0,0.0,0.0,1.0
416,3,NaN,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [127]:
preds = xg_reg.predict(OH_X_train)
binary = [round(x) for x in preds]
sum(binary)

364

In [128]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':binary})

In [129]:
sub = submission.rename(columns={0: "Survived"})
sub

,PassengerId,Survived
0,892,1
1,893,1
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,1
414,1306,1
415,1307,1
416,1308,1


In [131]:
filename = 'Titanic_02.csv'
sub.to_csv(filename,index=False)